# To generate spectrograms of augmented data

# Mount Google drive


In [ ]:
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


**Install** and load libraries

In [ ]:
!pip install -q fastai fastcore --upgrade # Make sure we have the new version
!pip install -q python_speech_features
!pip install python_speech_features

     |████████████████████████████████| 194kB 5.7MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 


In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
import IPython.display as ipd
import librosa
import python_speech_features as psf
from matplotlib import pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join
import os

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


**Read** the data

In [ ]:
# Load the augmented data

augmented = pd.read_csv('/content/drive/My Drive/ZINDI/train_augmented_data.csv')
augmented.head()
# Adjust name
# augmented['fn'] = augmented['fn'].str.replace('audio_train/', '/content/drive/My Drive/ZINDI/data/audio_augmented/')
# augmented.head()

,Unnamed: 0,fn,label
0,1,audio_train/aug-speed-IV38R7F.wav,akawuka
1,2,audio_train/aug-speed-KM4SKWT.wav,banana
2,3,audio_train/aug-speed-F5POSU9.wav,obulwadde
3,4,audio_train/aug-speed-MMVDXG2.wav,nnyaanya
4,5,audio_train/aug-speed-9TVM96F.wav,pampu


In [ ]:
augmented[:4710].tail()

,Unnamed: 0,fn,label
4705,4706,audio_train/aug-speed-audio_train/d027264654b94950aeb204f64d05d8d7_a39f129a06af41e7acb121a108ad5d8f.wav,sikungula
4706,4707,audio_train/aug-speed-audio_train/d027264654b94950aeb204f64d05d8d7_c4aecc275fad4309b83bd2f8c43d624d.wav,sikungula
4707,4708,audio_train/aug-speed-audio_train/d027264654b94950aeb204f64d05d8d7_f0aeba9bc00c4df5b39b2766c9dbd2fe.wav,sikungula
4708,4709,audio_train/aug-speed-audio_train/d027264654b94950aeb204f64d05d8d7_f5ce29d8dee84c83913066b9cde5bb41.wav,sikungula
4709,4710,audio_train/aug-shift-IV38R7F.wav,akawuka


# Get spectrograms

In [ ]:
# Define the function

def im_from_audio(fn, sample_rate=44100, window_length=0.05, window_step=0.0045, NFFT=2205):
  
  # Load the audio into an array (signal) at the specified sample rate
  signal, sr = librosa.load(fn, sr=sample_rate)

  # preemphasis
  signal = psf.sigproc.preemphasis(signal, coeff=0.95)

  # get specrogram
  # Get the frames
  frames = psf.sigproc.framesig(signal, 
                                  window_length*sample_rate, 
                                  window_step*sample_rate, 
                                  lambda x:np.ones((x,)))        # Window function 
    
  # magnitude Spectrogram
  spectrogram = np.rot90(psf.sigproc.magspec(frames, NFFT))
  
  # get rid of high frequencies
  spectrogram = spectrogram[512:,:]

  # normalize in [0, 1]
  spectrogram -= spectrogram.min(axis=None)
  spectrogram /= spectrogram.max(axis=None)        

  # Clip to max 512, 512
  spectrogram = spectrogram[:512, :512]
  
  return spectrogram 

Apply the function to the augmented data

In [ ]:
# Load all files and transform them into spectograms
for i in augmented.fn:
  interim1 = i.replace('.wav', '')
  interim2 = interim1.replace('/content/drive/My Drive/ZINDI/data/audio_augmented/', '') 
  name = '/content/drive/My Drive/ZINDI/data/spectrogram_augmented/' + interim2 + '-spectrogram.png'
  plt.imsave(fname = name, arr = im_from_audio(i))